In [1]:
# %pip install tensorflow_cpu==2.18.0

In [2]:
# %pip install keras


In [3]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
import tensorflow as tf
print(tf.__version__)


2.18.0


In [5]:
import pandas as pd
import numpy as np
import keras

import warnings
warnings.simplefilter('ignore', FutureWarning)

In [6]:
filepath='https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [7]:
concrete_data.shape

(1030, 9)

In [8]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [9]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [10]:
concrete_data_columns = concrete_data.columns

In [11]:
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [12]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [13]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [15]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [14]:
n_cols = predictors_norm.shape[1] # number of predictors

In [19]:
n_cols

8

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

In [17]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [20]:
# build the model
model = regression_model()

In [21]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100
23/23 - 1s - 38ms/step - loss: 1633.2893 - val_loss: 1117.2472
Epoch 2/100
23/23 - 0s - 5ms/step - loss: 1500.5846 - val_loss: 975.5190
Epoch 3/100
23/23 - 0s - 5ms/step - loss: 1272.9146 - val_loss: 755.9138
Epoch 4/100
23/23 - 0s - 5ms/step - loss: 930.1559 - val_loss: 481.3270
Epoch 5/100
23/23 - 0s - 5ms/step - loss: 553.0948 - val_loss: 260.3663
Epoch 6/100
23/23 - 0s - 5ms/step - loss: 313.7539 - val_loss: 177.4929
Epoch 7/100
23/23 - 0s - 5ms/step - loss: 244.1379 - val_loss: 164.8585
Epoch 8/100
23/23 - 0s - 5ms/step - loss: 221.1068 - val_loss: 167.0406
Epoch 9/100
23/23 - 0s - 5ms/step - loss: 207.9200 - val_loss: 164.7664
Epoch 10/100
23/23 - 0s - 5ms/step - loss: 197.1824 - val_loss: 161.7742
Epoch 11/100
23/23 - 0s - 5ms/step - loss: 188.3918 - val_loss: 158.9237
Epoch 12/100
23/23 - 0s - 5ms/step - loss: 181.3653 - val_loss: 157.8160
Epoch 13/100
23/23 - 0s - 5ms/step - loss: 175.8137 - val_loss: 155.1878
Epoch 14/100
23/23 - 0s - 5ms/step - loss: 168.8406 - v

In [22]:
def regression_model():
    input_colm = predictors_norm.shape[1] # Number of input features
    # create model
    model = Sequential()
    model.add(Input(shape=(input_colm,)))  # Set the number of input features 
    model.add(Dense(50, activation='relu'))  
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu')) 
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))  
    model.add(Dense(1))  # Output layer
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [23]:
model = regression_model()
model.fit(predictors_norm, target, validation_split=0.1, epochs=100, verbose=2)

Epoch 1/100
29/29 - 2s - 57ms/step - loss: 1461.7382 - val_loss: 846.8385
Epoch 2/100
29/29 - 0s - 4ms/step - loss: 489.5653 - val_loss: 203.1628
Epoch 3/100
29/29 - 0s - 4ms/step - loss: 231.6350 - val_loss: 183.1444
Epoch 4/100
29/29 - 0s - 5ms/step - loss: 196.9732 - val_loss: 177.6007
Epoch 5/100
29/29 - 0s - 5ms/step - loss: 183.2000 - val_loss: 156.6969
Epoch 6/100
29/29 - 0s - 5ms/step - loss: 164.0904 - val_loss: 146.8222
Epoch 7/100
29/29 - 0s - 6ms/step - loss: 148.7939 - val_loss: 128.4468
Epoch 8/100
29/29 - 0s - 5ms/step - loss: 132.9190 - val_loss: 119.5272
Epoch 9/100
29/29 - 0s - 5ms/step - loss: 114.5891 - val_loss: 107.1647
Epoch 10/100
29/29 - 0s - 4ms/step - loss: 98.4033 - val_loss: 99.0160
Epoch 11/100
29/29 - 0s - 4ms/step - loss: 83.2345 - val_loss: 89.9965
Epoch 12/100
29/29 - 0s - 5ms/step - loss: 74.9984 - val_loss: 81.1361
Epoch 13/100
29/29 - 0s - 4ms/step - loss: 65.2453 - val_loss: 74.1172
Epoch 14/100
29/29 - 0s - 4ms/step - loss: 58.6343 - val_loss: 76.

- Adding more hidden layers to the model increases its capacity to learn and represent complex relationships within the data. This allows the model to better identify, as a result, the model becomes more effective at fitting the training data and potentially improving its predictions.
- By reducing the proportion of data set aside for validation and using a larger portion for training, the model has access to more examples to learn from. This additional training data helps the model improve its understanding of the underlying trends, which can lead to better overall performance.